# The connection between Autoencoders and Matrix Factorization, and extension to link prediction

I'll start off by explaining the connection between autoencoders and matrix factorization, then go a bit further and explain how to use an autoencoder for link prediction in an undirected adjacency matrix, and conclude with a proposed way forward in the case of a directed adjacency matrix.

## Autoencoders and Matrix Factorization

In the paper put on arXiv in July 2016, (and currently under review), authors Florian Strub, Jérémie Mary, and Romaric Gaudel explain the relationship between autoencoders and matrix factorization. I thought it was a neat result. Here are some more details about the connection, in the context of recommender systems. 

Suppose we have $M$ movies and $N$ users. 

We know that each user $u_i$ could have rated a maximum of $M$ movies. Therefore $u_i \in \mathcal{R^{M}}$. We know, however, that for a sufficiently large database of movies, it's impossible for any one user to have seen them all. We're interested in predicting which movies user $u_i$ would rate highly, based on known ratings from movies they've watched.

The classical way to approach this problem is through matrix factorization. The goal is to retrieve information that can lead us to make good recommendations of movies to users. This information is encoded in the input (the vector of ratings associated with the user), and possibly some side information. The side information could be things like the user's current mood - not something that's captured explicitly already in the ratings. More on that later.

Say we have a matrix $R$ that has users as rows and movies as columns. Each user has only seen (and rated) a few of these movies. We have $N$ total users and $M$ total movies, so $R \in \mathcal{R}^{N {\rm{x}} M}$. Using matrix factorization, we will decompose this matrix $R$ into two separate matrices: $U \in \mathcal{R}^{N {\rm{x}} k}$, which encodes user information, and $V \in \mathcal{R}^{M {\rm{x}} k}$, which encodes movie information. These two matrices will be dense - that is, even though the original matrix of ratings $R$ has missing information, we're going to find two rank $k$ matrices $U$ and $V$ such that $\hat{R} = UV^T$.

The objective function for this technique is:

\begin{equation*}
{\rm{ argmin_{(U, V)}}} \sum_{(i, j) \in \mathcal{K} (R)} (r_{ij} - {\bar{u}_i}^T \bar{v}_j)^2 + \lambda\left( {\lVert {\bar{u}_i}^2 \rVert}_F + {\lVert{\bar{v}_j}^2 \rVert}_F \right)
\end{equation*}

where:
- $\mathcal{K}(R)$ is the set of known ratings
- $r_{ij}$ is the true (known) movie rating that user $i$ gave to movie $j$
- $\bar{u}_i$ is a dense, low-rank row of $U$ that encodes user information
- $\bar{v}_j$ is a dense, low-rank row of $V$ that encodes movie information
- $\lambda$ is a regularization parameter
- ${\lVert \cdot \rVert}_F$ is the Frobenius norm.




The matrices $  U$  and $  V$ are the optimal solutions to this objective function. This is essentially saying that we want to find a dot product between a user vector $  u_i$ and an item vector $  v_j$ that's as close as possible to what the true rating $  r_{ij}$ is, in the training set, in terms of a squared distance. We penalize the Frobenius norm of the vectors (a form of regularization) so that we can keep everything under control, and that penalty term has a `fudge factor' (or learning rate) $  \lambda$. The $  \lambda$ allows you to play with how much you want to penalize the size of $ u_i$ and $ v_j$. One weakness of this objective (I think - please feel free to correct me!) is that you have to control the sizes of these vectors together - the use of $ \lambda$ controls both norms at the same time. It might be more convenient to let more user information through, for example, and less item information, if you happen to have more user information than item information.

Now, matrix factorization is all well and good, but we might be able to do better. Recent work has highlighted the use of autoencoders in collaborative filtering. Stacked denoising autoencoders in particular have shown success. One interesting question is: how are matrix factorization and autoencoders related?

First, let's quickly review denoising autoencoders. 

Since I'm a statistics nerd, I'll link them to something stats-y: PCA. (As a side note, you could also think of matrix factorization as a deterministic version of mixed membership stochastic blockmodels, but that's a story for another day).

Traditionally, autoencoders are unsupervised, deterministic learning tools. They take an input $ x \in \mathcal{R}^{N}$ and perform dimensionality reduction to a hidden state $ h \in \mathcal{R}^{k}$, where $ k \ll N$. Then, they use this more compressed latent representation to reconstruct the input $ \hat{x} \in \mathcal{R}^{N}$. The point is data compression: if we can store the useful/informative features of $ x$ in a smaller space, then we save memory. 

Statisticians may notice a link to Principal Components Analysis (PCA) here, and there is one: a linear autoencoder accomplishes the same objective as PCA.
  
Statisticians often look at PCA as a way of summarizing the data by finding properties (or linear combinations of properties) that vary the most between observations. Another way to think of PCA is as a way to reconstruct your data, if you suddenly lost your dataset but retained the lower-dimensional information. It's this second point of view that provides the link to autoencoders. Statisticians would say we're maximizing the variance; machine learners would say we're minimizing the reconstruction error. The objective is exactly the same (want more details? <a href="http://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues">Look here</a>). PCA and autoenoders have many uses, any time data need to be projected into a lower dimensional space. They're used in image compression, for example, but today we're going to focus on the problem of finding the most important features (principal components!) that can reconstruct the user/movie pairs of ratings. We're also going to be talking about autoencoders with nonlinear transformations, so the connection with PCA is not preserved.

While the technique of matrix factorization allows us to discover some latent (hidden) features that contribute to a rating, we also want to know what to do when a rating is missing. Denoising autoencoders allow us to infer missing ratings in a more dynamic manner.

If we denote a neural network (the denoising autoencoder, in this case) by $ nn$, we can write the objective for the collaborative filtering task per Strub et al 2016 as:

\begin{equation*}
\mathcal{L}_{2, \alpha, \beta} (x, \tilde{x}) = \alpha \left( \sum_{j \in C(\tilde{x}) \cap K(x)} \left[ nn(\tilde{x})_j - x_j\right]^2 \right) + \beta \left( \sum_{j \notin C(\tilde{x}) \cap K(x)} \left[ nn(\tilde{x})_j - x_j\right]^2 \right) 
\end{equation*}

where:

 <ul>
  <li>$ x$ is some input</li>
  <li>$ \tilde{x}$ is a corrupted version of the input </li>
  <li>$ \alpha$ is a regularizing parameter to indicate to the network how much to focus on denoising the input</li>
  <li>$ \beta$ is a regularizing parameter to indicate to the network how much to focus on predicting missing input</li>
  <li>$ K(x)$ is the set of all known inputs</li>
  <li>$ C(\tilde{x})$ is the set of all corrupted inputs</li>
 <li>$ C(\tilde{x}) \cap K(x)$ is the set of all corrupted inputs for which we know the true input (the set of all deliberately corrupted inputs)</li>
 <li>$ nn(\tilde{x})$ is the output of the denoising autoencoder, when corrupted input $ \tilde{x}$ is provided</li>
</ul>



Notice this is accomplishing a mixture of unsupervised and supervised learning: the first term of the objective takes input that we have deliberately corrupted from the set of known input, and tries to reconstruct a clean version of the input. Since we know the true value of the input, this is the supervised portion of the objective. The second term takes missing items and tries to infer their value (the unsupervised portion). The last term in the objective is a regularizer. 

However, we don't have just one set of things to learn: we have both user and item information, which means we can use TWO denoising autoencoders - one to learn information about the users, and one for the items. This overcomes the issue I pointed out before about the regularization term applying to both the users and the items at the same time in matrix factorization, as there will be two separate autoencoders and thus two separate objectives. Unfortunately, it also means that we have double the work to do, so training may take longer.

So how are the objective functions for matrix factorization and this modified denoising autoencoder related? We need to go a little further into the math of what's happening with the denoising autoencoder to see what's really going on here.

If we provide an input $ u_i \in \mathcal{R}^{M}$ to a denoising autoencoder, then the model will transform that input to a latent space $ h \in \mathcal{R}^k$ via a nonlinear transformation: $h = \sigma(W_1 u_i + b_1)$, where $ \sigma$ could be a sigmoid function, for example, and $ W_1 \in \mathcal{R}^{M {\rm{x}} k}, b_1 \in \mathcal{R}^{k}$. Now, this latent representation preserves only the $ k$ most necessary pieces of information. We use these $ k$ pieces to reconstruct the input via another transformation. For the sake of argument, let's suppose it's a linear transform: $ W_2 h + b_2$, where $ W_2 \in \mathcal{R}^{k {\rm{x}} M}, b_2 \in \mathcal{R}^{M}$. We can write this in another way:

\begin{align}
    \hat{u}_i &= \underbrace{\left[W_2 \hspace{0.2cm} I_N \right]}_{V}  \underbrace{\begin{bmatrix}
           \sigma(W_1 u_i + b_1)  \\
           b_2 \\
         \end{bmatrix}}_{\bar{u}_i}
  \end{align}
  
So that we learn a matrix transformation from the hidden space to the output vector: $ f: \mathcal{R}^{k} \to \mathcal{R}^{M}$. We thus learn a matrix $ V = W_2 \in \mathcal{R}^{k {\rm{x}} M}$. Now we have a reconstructed version $ \hat{u}_i$ of our input $ u_i$, which has been denoised and filled in as necessary.

We can do the same thing, without loss of generality, for the movies, learning a matrix $ U$. 

The objective for the matrix factorization routine has a dot product between the user information and the item information, and is trying to find two matrices $ U$ and $ V$ to make the result as close as possible to the true rating.

The objective for Strub et al 2016's denoising autoencoder looks directly at the input, and constructs two separate autoencoders: one for the users and one for the movies. If you were to separate out what's going on in the matrix factorization objective, it would look a lot like the autoencoder, especially when you consider the linear transformation that is the output of the neural network.

So that's the first part of the story! Autoencoders and matrix factorization are really closely linked, and provide a nice transition to deep recommender systems.



## Using autoencoders for link prediction

Now that we've covered some basic ground on autoencoders and matrix factorization, how can we reformulate this task specifically for link prediction?

The link prediction problem can be formulated as a matrix completion problem on the adjacency matrix of a graph. As before, when the adjacency matrix is sparse, this is a difficult problem. We need to have a model sophisticated enough to fill in the missing values, without imputing links that never would have existed, using very little information.

### Let's step back a moment: what exactly is a graph, and what is an adjacency matrix?

A graph $G = \left\{V, E\right\}$ is a set of vertices (or nodes) $V$ and edges (links) $E$. The edges connect none, some, or all of the vertices. Each link is either present or absent, so we have a 0/1 relationship between vertices. We can extend this to include a weight on each edge, that tells us the strength of the connection. We could go so far as to represent a rating matrix as a bipartite graph between users and items, and the weight on each edge would be the rating itself. 

In this case, to start, we will focus on the existence of a link, and interpret that existence as 'user i would be interested in item j'. For simplicity of exposition, we'll also restrict our methodology to work on a symmetric matrix: that is, the number of users is the same as the number of items (for now).

The adjacency matrix describes which vertices have links and which do not. For example, suppose there were 3 users and 3 items. Then if user 1 is interested in items 2 and 3, and user 2 is interested in item 1, and user 3 is interested in items 1 and 3, then the adjacency matrix $A$ would look like this:

$$\left(\begin{array}{ccc} 
 0 & 1 & 1\\
 1 & 0 & 0\\
 1 & 0 & 1\\
\end{array}\right)$$

### Hold on, how does this relate to matrix factorization?

As was said before, though, the adjacency matrix is often large and sparse. The problem becomes inferring relationships. Matrix factorization can fill in this adjacency matrix. The objective function looks a little different from what we've seen:

\begin{equation*}
{\rm{min}} \sum_{i, j} \mathcal{L}(A_{ij}, g(W_{2, j}W_{1, i} + b_{1, i} + b_{2, j}))
\end{equation*}

where 
- $A_{ij}$ is the ${ij}^{th}$ entry of the adjacency matrix
- $W_2 \in \mathcal{R}^{N {\rm{x}} K}, W_1 \in \mathcal{R}^{K {\rm{x}} N}, b_1 \in \mathcal{R}^{N}, b_2 \in \mathcal{R}^{N}$
- $W_{2, j}$ is the $j^{th}$ row of $W_2$
- $W_{1, i}$ is the $i^{th}$ row of $W_1$
- $g$ is a link function
- $\mathcal{L}$ is a loss function

Connecting this back to the first discussion, this matrix factorization objective is exactly the same as Strub et al 2016's. They just expressed it a little differently. If we let $A_{ij} = r_{ij}$, the ${ij}^{th}$ entry of the ranking matrix, then we're doing a dot product between two vectors (the W's in the equation above), trying to get the best reconstruction of $A^{N {\rm{x}} N}$. If we add a regularization constraint, assume an L2 loss function, and assume we're only looking at known entries in the adjacency matrix, then this new objective function matches Strub et al 2016's:

\begin{equation*}
{\rm{ argmin_{(U, V)}}} \sum_{(i, j) \in \mathcal{K} (R)} (r_{ij} - {\bar{u}_i}^T \bar{v}_j)^2 + \lambda\left( {\lVert {\bar{u}_i}^2 \rVert}_F + {\lVert{\bar{v}_j}^2 \rVert}_F \right)
\end{equation*}

### Now to move on to autoencoders.

Our feature vectors $x_i$ are going to be the set of known neighbours of $x_i$. That is, we will set $x_i = A_i$, instead of looking at user vectors $u_i$ and $v_j$, which encoded all ratings made by user i and all ratings of item j, respectively, in the previous discussion.

The loss function for autoencoders can be written as:

\begin{equation*}
 {\rm{min}} \sum_{i} \mathcal{L}(\tilde{x}_i, g(W_2(f(W_1 x_i + b_1)) + b_2))
\end{equation*}

The general goal is to minimize the loss between the reconstruction $\tilde{x}$ and the input $x$. To make the connection between matrix factorization and autoencoders more concrete, consider the case $f=I$, the identity mapping. Then the autoencoder is learning a linear mapping from the input to its reconstruction. However, it also means that learning another layer of weights is redundant: a linear mapping of a linear mapping is itself a linear mapping, so the entire operation could be reduced to a single linear operation. 

One way around this inability to learn a meaningful deep representation is to *corrupt* the input, as discussed earlier with the denoising autoencoder. We can only feed in the input that we know: we need to infer a dense vector from a sparse one. To make the best use of the information we have, it makes sense to force the autoencoder to learn to denoise the input as well as to reconstruct it. We can deliberately mask some of the input, and the objective will either denoise or reconstruct, depending on whether the input element was masked or not. 

This also makes sense from the perspective of the structure of an autoencoder. If there are many missing or masked input elements, then the bottleneck (the dimensionality of the smallest hidden layer) may have more entries than the input vector, which may not be a problem (after all, an overcomplete representation can be useful), but now instead of just reconstructing the input, the autoencoder will also attempt to fill in the missing values. By changing the objective function to do this explicitly, Strub et al 2016 show the connection to link prediction without really saying so. Here's their objective function again, for reference:

\begin{equation*}
\mathcal{L}_{2, \alpha, \beta} (x, \tilde{x}) = \alpha \left( \sum_{j \in C(\tilde{x})} \left[ nn(\tilde{x})_j - x_j\right]^2 \right) + \beta \left( \sum_{j \notin C(\tilde{x})} \left[ nn(\tilde{x})_j - x_j\right]^2 \right) 
\end{equation*}

Zhai and Zhang 2015 go further to use both an autoencoder and matrix factorization jointly to fill in the missing values of the adjacency matrix. As they are predicting binary output, they use a cross-entropy loss function, instead of Strub et al 2016's L2 loss. Interestingly, they also argue that dropout is a better form of regularization for autoencoders than an L2 norm. This also makes sense given the structure of what they're trying to predict (the 0/1 output of presence/absence of a link). I still think the L2 norm regularization used by Strub et al 2016 makes sense for their reconstruction and denoising of real-valued input.

Chang and Zhang 2014 add side information as covariates, in contrast to the approach taken by Strub et al 2016, who append the side information to every layer of the network (extending both the input and the hidden layers, see figure below). 

<img src="Strubetal2016.png" alt="Adding side information" style="width: 350px;"/>

## Moving to directed links: substitutable vs complementary products

Through matrix factorization, or denoising autoencoders, we have a way of filling in a rating matrix between users and items, or an adjacency matrix among users or among items (as an adjacency matrix is formulated to be square).

However, we're more interested in the relationship between items. Specifically, we want to know which products are substitutes (buy product $i$ instead of product $j$) and which are complements (buy product $i$ in addition to product $j$). 

We can define an adjacency matrix between all products $i$ and $j$, such that $A_{ij} = 1$ if products $i$ and $j$ are related in some way, and $A_{ij} = 0$ otherwise. Denoising autoencoders can be used to fill in the missing links in this matrix.

But substitutes and complements are more well-defined by the direction in which a user would move, as stated in McAuley et al 2015.

For example, if product $i$ is cheaper than product $j$, but both are t-shirts, then a user would likely be more inclined to purchase product $i$ than product $j$. So the direction of the relationship between $i$ and $j$ flows from $j$ to $i$, and we get that information from the manifest feature 'price'. T-shirt $i$ is a good substitute for t-shirt $j$.

For a complement, the user could go in either direction - you can buy a t-shirt and then jeans, or jeans and then a t-shirt. So the information flow can go from product $i$ to product $j$ or vice versa, so we would expect that the information content would be the same in both directions. We would find some evidence to support the fact that the 'distance' between products $i$ and $j$ in that case should be symmetric. 

We want to detect which products are substitutes and which are complements.

We can start with substitutes. We want to find the adjacency matrix $A$, which only encodes which products are related (not how they are related) and then from there, figure out the direction of flow.

To do that, we could use a technique shown in this paper: Inferring Causal Direction from Relational Data (UAI 2016). In the case of a denoising autoencoder, we have a linear mapping from the input to the hidden (feature) space, and then a nonlinear mapping from the hidden space to the output which gives the reconstruction and predicted values for anything missing in the input. In Zhai and Zhang 2015 (available <a href="http://arxiv.org/pdf/1512.04483">here</a>), the authors feed in the set of known neighbours of the input, and infer the unknown neighbours. In Strub et al 2016 (<a href="https://arxiv.org/pdf/1603.00806">here</a>), the authors feed in either a vector of ratings from the user, or a vector of movie ratings for a movie, and fill in the missing values. Since we're more interested in the relationship between products, the former technique sounds more useful for now. However, we don't have any known neighbours to start out with. 

In our case, if we wanted to find out the neighbours of a product, we could start out with the category tree from Amazon, and say that two items are neighbours if they are in the same node of the category tree, and find other neighbours using the denoising autoencoder.

If we instead wanted to continue with users and movie ratings example, we could feed in movies that we know are in the same category (all action movies, for example), and do the same. The problem there, of course, is that we'd be very likely to just find more action movies, instead of finding movies with, for example, the same actor, but different genres (Keanu Reeves was in The Matrix, but was also in The Lake House, which are completely different movies, but are related through Keanu Reeves). So instead, it would make more sense to enter a movie and discover useful/salient features of the movie, then feed that into a logistic regression output to just say 1/0 (yes/no) whether two movies are related.

So my idea is to first find the adjacency matrix, then discover direction, if it exists, from there. Arbour et al 2016 discuss non-symmetric distances between people based on manifest features. Instead, I'd be looking at non-symmetric distances between products based on manifest AND discovered/learned features. They talk about mapping a probability distribution to a Hilbert space, and looking at distances via two kernel functions - one to map the distribution to the space, and then one to measure (non-symmetric) distance between the means of the distributions. 

Based on that idea, I will instead use KL divergence, since I'm looking at a non-symmetric 'distance measure', and work in Riemannian space instead of a Hilbert space. It requires considering the input vectors as samples from a probability distribution. It would have to be a discrete distribution, though, because you can't change 't-shirt' by 0.01 and say it's a new product. Intuition says it should be a categorical distribution. 

The products would be represented as vectors encoding their category, and we'd be inferring, via the hidden layer and some manifest features, what makes products related to one another. Then we would measure the KL divergence between the hidden representations between two products, and if KL(product $i$ || product $j$) $>>$ KL (product $j$ || product $i$), then we would say the user is more likely to go from product $i$ to product $j$ (they're a bit more different: product $j$ is significantly cheaper, for example, than product $i$). If instead KL(product $i$ || product $j$) = KL(product $i$ || product $j$) + $\epsilon$, for some small epsilon, then we'd say there's no clear direction of information flow, and the products *may* be complements, but are definitely not substitutes.

Therefore, by computing the adjacency matrix, I can use the hidden features in the autoencoder, as well as the manifest features, to infer direction.

If we were instead interested in ratings:

Suppose we have a user rating vector $u_i \in R^{M}$, which is a sample from a random variable $U_i$, and a vector of movie ratings $v_j \in R^{N}$ as a sample from a random variable $V_j$. 

Then we would be interested in minimizing the KL divergence between any known ratings $r_{ij}$, which are essentially samples from the 'true posterior distribution', $R_{ij}$ and the dot product $u_i * v_j$, which can be viewed as a sample from the 'approximate posterior' $U_i*V_j$, which we could argue follows a particular distribution (like a Normal distribution, as done by Salakhutdinov et al in Probabilistic Matrix Factorization). Instead of using a symmetric function like the squared error, we'd use a KL divergence. We can use variational inference to find parameters of $U_i*V_j$ that minimize the KL divergence between the approximate and true posteriors. Once we've found those parameters, we now have a good approximator to the true posterior distribution, and we could sample from the good approximate posterior $U_i*V_j$ to recommend products to users.

Recall that McAuley et al 2015 performed supervised link prediction, in that they assumed that of their four available graphs, two gave some notion of 'substitute', and two were associated with complements. Their philosophy on that subject makes some intuitive sense, and we can build on it.

Formalizing this, the objective function becomes:

$\mathcal{L}(y, d \mid W_1, W_2, A_i, b_1, b_2, \theta) = \prod_{g \in G} \left\{ \prod_{i \in V_g}  {\rm{min}} \left(\sum_i L\left[ A_i, g(W_2(f(W_1A_i + b_1) + b_2))\right]\right) {\rm{max}} \left[ KL([h, \theta]_i, [h, \theta]_j), KL([h, \theta]_i, [h, \theta]_j)\right] \right\}$

where:
   - $y$ is 0 if products $i$ and $j$ are unrelated, 1 otherwise
   - $d$ is the direction of the relationship between $i$ and $j$
   - $W_1$ is the weight matrix from the input to the hidden variables in a denoising autoencoder
   - $W_2$ is the weight matrix from the hidden to the reconstructed and predicted input (the filled adjacency matrix) in a denoising autoencoder    
   - $A_i$ is the $i^{th}$ row of the adjacency matrix (the neighbours of product $i$)
   - $b_1$ is the bias on the hidden units
   - $b_2$ is the bias on the output units (the reconstructed and predicted input)
   - $\theta$ are the manifest features associated with product $i$ (price, colour, etc.)
   - $g \in G$ indexes the four graphs    
   - $i \in V_g$ indexes the vertices (nodes, products) in the $g^{th}$ graph
   - $L$ is a loss function, taken to be the cross-entropy
   - $g$ is a link function, taken to be the identity
   - $f$ is a nonlinearity, taken to be sigmoid
   - $KL$ is a Kullback-Leibler divergence
   - $[h, \theta]_i$ is the vector of discovered hidden features of product $i$, appended with its manifest features

We are trying to minimize the cross-entropy error in the denoising autoencoder, and using the direction of maximum KL divergence to inform whether the relationship flows from $i$ to $j$ or from $j$ to $i$.

[EDIT] Note that directed edges do not just mean products are substitutable. For example, we may be interested in recommending a desk lamp to someone who bought a desk, but we wouldn't recommend a desk to someone who bought a desk lamp. Therefore complementary products can also have an associated direction. This is already taken care of in the objective function, as we're taking a product over all graphs, and we know which graphs have substitutable and which have complementary products.

## Side comment: how the category tree is used in McAuley et al 2015

The authors want to create a sparse hierarchy of topics, as training a model with many (hundreds) of topics per product is not realistic. 

If we start with side information (the category tree), a product is represented as a path through the category tree. 

The root node starts with 10 topics that are generic across all categories. From there, a topic is added to each node in the category tree. For example, a laptop charger is represented as 'Electronics -> Computers -> Laptops -> Accessories -> Chargers'. The authors associate one topic with each of these levels in the category tree. So 'Electronics' starts with 1 topic; 'Computers' starts with 1 topic, etc. 

Each topic has a node. When they observe a certain number of instances of a node, they associate a new topic with that node. For example, when they've seen 1000 instances of 'Computer', they give the node 'Computer' another topic. But if 'Computer' gets another topic, then the previous nodes in the hierarchy must also get a new topic, so 'Electronics' gets another topic. 

As another example, if 'Computer' and 'Electronics' now both have two topics, and we observe 1000 instances of 'Laptops', then 'Laptops' would get a second topic, and 'Computers' and 'Electronics' would each get a new topic. The final setting after these changes would be: 'Computers': 3 topics; 'Electronics': 3 topics; 'Laptops': 2 topics; 'Accessories': 1 topic; 'Chargers': 1 topic. 

In this way, the coarsest level of granularity is always associated with the most topics, which makes intuitive sense. The authors are also able to use this type of system to discover 'micro-categories': categories with finer levels of granularity than the category tree itself. For example, if 1000 instances of 'Chargers' are seen, then a new topic is associated with 'Chargers'. But there's no further level of granularity to reach in the category tree, so when a new topic is added, that serves as a pseudo-extension of the category tree. 

For example, there might be 'Apple chargers' and 'non-Apple chargers' discovered at first (with this extra topic) and if 1000 more instances of 'Chargers' are observed, and a third topic is thus added, then maybe this third topic would capture something regarding longevity, or another brand. In fact, the authors find that micro-categories are often capturing something to do with brand information.

### Comments in the paper that support my argument:

"Each topic is associated with a node in Amazon's category tree" (section 4.6).
"The topics we obtain are closely aligned with categories from Amazon...though this is to be expected since our topic model is built on top of an explicit [category] hierarchy....However, we note that finer-grained 'micro categories' are discovered that are not explicitly present in the [category] hierarchy, e.g. high-end headphones are distinguished from cheaper models....We also note that brand words predominate in several topics, e.g. high-end headphones can be identified by words like 'Sennheiser, 'AKG', etc." 
"[Generic] topics tend to appear toward the top of the category hierarchy"

### Comments in the code that support my argument:

features.cpp: "create the tree of topic IDs from the category IDs in the corpus" (CA: so the number of topics comes from the number categories)

categoryTree.cpp: "the parent always has more topics than the child" (CA: so when we increment the number of topics associated with a child node, we must increment the number of topics associated with a parent node. Unless the set of topics associated with the parent node by definition includes the number of topics associated with the child nodes?)

categoryTree.cpp: "The number of topics associated with a category is based on the number of times the category was observed in the corpus. Start with 10 'generic' topics that belong to the root node (that are used by all products). Once we observe a fixed number 'productsPerTopic' instances of a node in the category tree, add a topic to that node, up to a maximum of 10 topics per sub-category (CA: per NODE). 

categoryTree.cpp: "Increment all nodes along a path (e.g. for a product in Electronics-> Mobile Phones -> Accessories increment all three category nodes". (CA: Since the number of topics associated with a category depends on the number of instances of the category in the corpus, every time we observe 1000 (productsPerTopic) instances of Mobile Phones, we will increase the number of topics associated with Mobile Phones by 1 and the number of topics in Electronics by 1.)

### Last notes
    
The category tree informs the topic distribution. In section 3.2.3, the authors state that they only allow a product review to draw words from topics that are part of its path in the category tree. That is, say we have a laptop charger. The laptop charger's review must be composed of words that came from topics that are in Electronics, Computers, Laptops, Accessories, and Chargers. It may not use topics that are part of another path in the tree, such as clothing. This makes some intuitive sense.

In the likelihood function, the topic distribution for the $j^{th}$ word of document (review) $d$ is represented as $\theta_{z_{d, j}}$. These topics are associated with nodes in the category tree. Therefore any part of the likelihood function that uses the topics is using the category tree. The topics come into play in many aspects of the likelihood function, as $\phi$ and $\varphi$ are both functions of $\theta$.


## References

Arbour, D. et al. Inferring Causal Direction from Relational Data. Proceedings of Uncertainty in Artificial Intelligence, 2016. [Online]: http://auai.org/uai2016/proceedings/papers/217.pdf

Chen, Z. and Zhang, W. A Marginalized Denoising Method for Link Prediction in Relational Data. SIAM International Conference on Data Mining 2014. [Online]: http://epubs.siam.org/doi/abs/10.1137/1.9781611973440.34

McAuley, J. et al. Inferring Networks of Substitutable and Complementary Products. KDD 2015. arXiv: http://arxiv.org/abs/1506.08839

Strub, F. et al. Hybrid Collaborative Filtering with Autoencoders. Deep Learning Summer School 2016 [poster]. arXiv: http://arxiv.org/abs/1603.00806v3


Zhai, S. and Zhang, Z. Dropout Training of Matrix Factorization and Autoencoder for Link Prediction in Sparse Graphs. arXiv: http://arxiv.org/abs/1512.04483

